In [2]:
import os
os.chdir('..')

In [27]:
import pandas as pd
import numpy as np
from src.construct import util

In [4]:
nat_annual = util.read_tdf('build/national/annual-demand.csv')
reg_annual = util.read_tdf('build/eurospores/annual-demand.csv')

In [5]:
reg_annual_grouped = reg_annual.unstack('id')
reg_annual_grouped.columns = reg_annual_grouped.columns.str.split('_', expand=True)
reg_annual_grouped = (
    reg_annual_grouped
    .groupby(level=0, axis=1).sum()
    .rename_axis(columns='id')
    .stack()
    .reorder_levels(nat_annual.index.names)
)

In [8]:
all_annual = pd.concat([nat_annual, reg_annual_grouped], keys=['national', 'regional'], axis=1, sort=True)

In [12]:
deviations = (all_annual.national - all_annual.regional)
print(f"maximum deviation between national and regional data is {deviations.max()}")

maximum deviation between national and regional data is 7.450580596923828e-09


In [13]:
deviations.idxmax()

('transport_vehicles', 'road', 2001.0, 'ITA', 'vehicles', 'passenger_car')

# Test failing sections in national scale model

In [50]:
space_heat = "build/model/national/space-heat-bau-electricity-demand.csv"
water_heat = "build/model/national/water-heat-bau-electricity-demand.csv"
cooking = "build/model/national/cooking-bau-electricity-demand.csv"
public_transport = "build/national/public-transport-demand.csv"
annual_demand = "build/national/annual-demand.csv"
hourly_electricity = "euro-calliope/build/model/national/electricity-demand.csv"
model_year = 2016

profiles = {  # all profiles are negative values
    k: pd.read_csv(v, index_col=0, parse_dates=True)
    for k, v in {
        'space_heat': space_heat, 'water_heat': water_heat,
        'cooking': cooking, 'transport': public_transport,
    }.items()
}
hourly_electricity_df = pd.read_csv(hourly_electricity, index_col=0, parse_dates=True)
print("There are {} missing data points, which will be backfilled".format(hourly_electricity_df.isna().sum().sum()))
hourly_electricity_df = hourly_electricity_df.bfill()
new_hourly_electricity_df = hourly_electricity_df.copy()
annual_demand_df = util.read_tdf(annual_demand)
annual_demand_df = annual_demand_df.xs(model_year, level='year').droplevel('unit')
bau_electricity = (
    annual_demand_df
    .filter(regex='bau_electricity')
    .rename(lambda x: x.replace('_bau_electricity', '').replace('bau_', ''), level='end_use')
)
# Subtract old demand
# Industry is a flatline and is 'added' since hourly electricity is negative
new_hourly_electricity_df = new_hourly_electricity_df.add(
    bau_electricity
    .xs('industry_demand')
    .sum(level='id')
    .div(len(new_hourly_electricity_df.index))
)
# space heating, water heating, cooking, and non-freight transport is based on profiles
# We use rail profiles for all non-freight transport, which is a simplification in light
# of the small relative constribution of non-rail transport to electricity demand
for profile_name, profile in profiles.items():
    if profile_name == 'transport':
        scaled_profile = profile.apply(lambda x: x.div(x.abs().sum()))
        new_hourly_electricity_df = new_hourly_electricity_df.sub(
            scaled_profile
            # fillna for regions with no rail transport, but with other transport electricity demand
            .fillna({i: scaled_profile.mean(axis=1) for i in profile.columns})
            .mul(bau_electricity.xs('transport_demand').sum(level='id'))
        )
    else:
        new_hourly_electricity_df = new_hourly_electricity_df.sub(profile)
# Add back in new demand
# Industry, this time 'subtracted' to make hourly electricity more negative
new_hourly_electricity_df = (
    new_hourly_electricity_df.sub(
        annual_demand_df
        .xs('industry_demand')
        .xs('electricity', level='end_use')
        .sum(level='id')
        .div(len(new_hourly_electricity_df.index))
    )
)
# Rail transport (profile is negative, so we add it)
new_hourly_electricity_df = new_hourly_electricity_df.add(profiles['transport'])

There are 1 missing data points, which will be backfilled


In [54]:
(new_hourly_electricity_df.sum() -
        bau_electricity.sum(level='id') +
        annual_demand_df.xs('electricity', level='end_use').sum(level='id'))

ALB     -71.072234
AUT    -702.022110
BEL    -850.232770
BGR    -376.997119
BIH    -122.212334
CHE    -627.520630
CYP     -45.554092
CZE    -653.057100
DEU   -5162.525660
DNK    -339.700948
ESP   -2504.051190
EST     -81.428898
FIN    -850.583299
FRA   -4800.591600
GBR   -3760.377861
GRC    -512.127550
HRV    -175.973510
HUN    -418.893323
IRL    -269.275316
ISL    -181.182929
ITA   -3143.113550
LTU    -114.326920
LUX     -64.963088
LVA     -72.073300
MKD     -74.152670
MNE     -32.521578
NLD   -1145.305860
NOR   -1323.289838
POL   -1560.129316
PRT    -492.320047
ROU    -535.824530
SRB    -390.062900
SVK    -287.919387
SVN    -138.003030
SWE   -1393.658730
dtype: float64

In [59]:
np.isclose(
        hourly_electricity_df.sum(),
        new_hourly_electricity_df.sum()
        .sub(bau_electricity.sum(level='id'))
        .add(annual_demand_df.xs('electricity', level='end_use').sum(level='id'))
        .reindex(hourly_electricity_df.columns)
    )

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [60]:
new_hourly_electricity_df.where(lambda x: x > 0).dropna(axis=1, how='all').count()

CYP    207
dtype: int64